In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 


In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, SimpleRNN, MaxPooling1D, Conv1D
from keras.optimizers import RMSprop, Nadam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras import backend as K
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import  classification_report

import tensorflow as tf


np.random.seed(42)
tf.random.set_seed(42)

In [3]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [4]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix,accuracy_score

def imprimir_metricas(precisao, revocacao, f1, acuracia):
    precisao = precisao * 100
    revocacao = revocacao * 100
    f1 = f1 * 100
    acuracia = acuracia * 100
    print('precisao {:.2f}'.format(precisao))
    print('revocacao {:.2f}'.format(revocacao))
    print('f1-score {:.2f}'.format(f1))
    print('acuracia {:.2f}'.format(acuracia))

In [5]:
def model_SimpleRNN():
    model = Sequential()
    model.add(SimpleRNN(units=40))
    model.add(Dropout(0.5))
    model.add(Dense(units=1, activation='sigmoid'))
    return model
    
def model_LSTM():
    model = Sequential()
    model.add(LSTM(units=300, activation='sigmoid', return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(units=1, activation='sigmoid'))
    return model

def model_Bidirectional():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=300, activation='sigmoid', return_sequences=False)))
    model.add(Dense(1, activation='sigmoid'))
    return model

def model_CNN_LSTM():
    model = Sequential()
    
    # Convolutional Layer
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))
    # LSTM Layer
    model.add(LSTM(units=300, activation='sigmoid', return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [6]:
def plotGraph_models(model):
    history = model.history

    print(history.history.keys())

    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)


    plt.plot(epochs, acc, 'bo', label = 'Training Accuracy')
    plt.plot(epochs, val_acc, 'r', label = 'Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    y_arrow = max(val_acc)
    x_arrow = val_acc.index(y_arrow) + 1
    plt.annotate(str(y_arrow)[:6],
                (x_arrow, y_arrow),
                xytext=(x_arrow + 5, y_arrow + .02),
                arrowprops=dict(facecolor='orange', shrink=0.05))
    plt.xticks(epochs)

    plt.figure()
    plt.plot(epochs, loss, 'bo', label = 'Training Loss')
    plt.plot(epochs, val_loss, 'r', label = 'Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.xticks(epochs)
    plt.show()

In [7]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)
epochs = 20

In [8]:
periodo = 'pre-pandemia'

choq = ['volumeTweets_media', 'volumeTweets_variancia', 'volumeTweets_mediaMovelPonterada', 'volumeTweets_entropia', 
 'indiceInsonia_mediaMovelPonterada', 'indiceInsonia_entropia', 
 'pronome1Pessoa_media', 'pronome1Pessoa_variancia', 'pronome1Pessoa_mediaMovelPonterada', 'pronome1Pessoa_entropia', 
 'pronome2Pessoa_mediaMovelPonterada', 
 'pronome3Pessoa_variancia', 'pronome3Pessoa_mediaMovelPonterada', 'pronome3Pessoa_entropia', 
 'valencia_variancia', 'valencia_entropia', 
 'ativacao_variancia', 'ativacao_entropia', 
 'termosDepressivos_mediaMovelPonterada', 
 'grafoSocial_variancia', 'grafoSocial_mediaMovelPonterada', 'grafoSocial_entropia',
 'medicamentosAntiDepressivo_variancia'
 ]


atributos_anterioes = ['caracteresOrientais_variancia', 
 'emojis_mediaMovelPonterada', 
 'curtidas_media', 'curtidas_variancia', 'curtidas_mediaMovelPonterada',
 'midia_variancia',
 'links_entropia',
 ]

atributos_novos = ['hashtags_variancia', 'hashtags_mediaMovelPonterada', 
                   'retweets_media', 'retweets_variancia', 'retweets_mediaMovelPonterada', 
 'mencoes_media', 'mencoes_variancia', 'mencoes_mediaMovelPonterada', 
 'polaridade_entropia', 
 'subjetividade_entropia']

df = pd.read_csv(f'datasets/twitterbase_{periodo}.csv', sep=';')

cv = KFold(n_splits=10, shuffle=True, random_state=42)
zscore = StandardScaler()

train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

label_encoder = LabelEncoder()
df['classe'] = label_encoder.fit_transform(df['classe']).astype('float64')

df1 = df[choq + ['classe']].copy()
df2 = df[choq + atributos_anterioes + ['classe']].copy()
df3 = df[choq + atributos_novos + ['classe']].copy()

x1 = df1.drop('classe', axis=1).copy()
y1 = df1['classe'].copy()
x1 = zscore.fit_transform(x1)

x2 = df2.drop('classe', axis=1).copy()
y2 = df2['classe'].copy()
x2 = zscore.fit_transform(x2)

x3 = df3.drop('classe', axis=1).copy()
y3 = df3['classe'].copy()
x3 = zscore.fit_transform(x3)

model1 = model_SimpleRNN()
model2 = model_LSTM()
model3 = model_Bidirectional()
model4 = model_CNN_LSTM()

x4 = df.drop('classe', axis=1).copy()
y4 = df['classe'].copy()
x4 = zscore.fit_transform(x4)

# Simulação 1

In [9]:
x_train, x_val, y_train, y_val = train_test_split(x1, y1, test_size=validation_ratio, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_ratio, random_state=42)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 1)

In [10]:
model1.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model1.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model1.evaluate(x_val, y_val, verbose=0)

print()
print('SimpleRNN')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)





SimpleRNN
precisao 75.54
revocacao 75.68
f1-score 75.01
acuracia 74.23


In [11]:
model2.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model2.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model2.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM
precisao 72.38
revocacao 81.74
f1-score 76.30
acuracia 74.01


In [12]:
model3.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model3.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model3.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM Biredicional')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM Biredicional
precisao 76.71
revocacao 69.58
f1-score 72.40
acuracia 72.89


In [13]:
model4.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model4.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model4.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM + CNN')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)



LSTM + CNN
precisao 73.89
revocacao 78.54
f1-score 75.61
acuracia 74.10


# Simulação 2

In [14]:
x_train, x_val, y_train, y_val = train_test_split(x2, y2, test_size=validation_ratio, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_ratio, random_state=42)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 1)

In [15]:
model1.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model1.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model1.evaluate(x_val, y_val, verbose=0)

print()
print('SimpleRNN')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


SimpleRNN
precisao 75.79
revocacao 76.24
f1-score 75.44
acuracia 74.71


In [16]:
model2.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model2.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model2.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM
precisao 75.36
revocacao 72.34
f1-score 73.25
acuracia 72.99


In [17]:
model3.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model3.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model3.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM Biredicional')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM Biredicional
precisao 75.56
revocacao 77.13
f1-score 75.81
acuracia 74.84


In [18]:
model4.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model4.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model4.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM + CNN')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM + CNN
precisao 67.41
revocacao 95.19
f1-score 78.51
acuracia 73.18


# Simulação 3

In [19]:
x_train, x_val, y_train, y_val = train_test_split(x3, y3, test_size=validation_ratio, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_ratio, random_state=42)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 1)

In [20]:
model1.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model1.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model1.evaluate(x_val, y_val, verbose=0)

print()
print('SimpleRNN')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


SimpleRNN
precisao 71.18
revocacao 90.30
f1-score 79.19
acuracia 75.55


In [21]:
model2.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model2.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model2.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM
precisao 75.43
revocacao 79.52
f1-score 76.90
acuracia 75.62


In [22]:
model3.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model3.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model3.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM Biredicional')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM Biredicional
precisao 69.12
revocacao 93.27
f1-score 78.98
acuracia 74.52


In [23]:
model4.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model4.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model4.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM + CNN')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM + CNN
precisao 69.26
revocacao 93.10
f1-score 79.02
acuracia 74.57


# Simulação 4

In [24]:
x_train, x_val, y_train, y_val = train_test_split(x4, y4, test_size=validation_ratio, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_ratio, random_state=42)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 1)

In [25]:
model1.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model1.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model1.evaluate(x_val, y_val, verbose=0)

print()
print('SimpleRNN')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


SimpleRNN
precisao 67.65
revocacao 93.75
f1-score 78.16
acuracia 73.02


In [26]:
model2.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model2.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model2.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM
precisao 63.14
revocacao 97.00
f1-score 76.09
acuracia 68.60


In [27]:
model3.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model3.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model3.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM Biredicional')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM Biredicional
precisao 69.31
revocacao 93.54
f1-score 79.20
acuracia 74.76


In [28]:
model4.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model4.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model4.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM + CNN')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM + CNN
precisao 73.44
revocacao 85.42
f1-score 78.55
acuracia 76.05


# Pandemia

In [29]:
periodo = 'pandemia'

choq = ['volumeTweets_media', 'volumeTweets_variancia', 'volumeTweets_mediaMovelPonterada', 'volumeTweets_entropia', 
'indiceInsonia_variancia', 'indiceInsonia_mediaMovelPonterada', 'indiceInsonia_entropia', 
'pronome1Pessoa_media', 'pronome1Pessoa_variancia', 'pronome1Pessoa_mediaMovelPonterada', 'pronome1Pessoa_entropia', 
'pronome2Pessoa_mediaMovelPonterada', 
'pronome3Pessoa_media', 'pronome3Pessoa_variancia', 'pronome3Pessoa_mediaMovelPonterada', 'pronome3Pessoa_entropia', 
'valencia_mediaMovelPonterada', 'valencia_entropia', 
'ativacao_mediaMovelPonterada', 'ativacao_entropia', 
'termosDepressivos_variancia', 
'grafoSocial_variancia', 'grafoSocial_mediaMovelPonterada', 'grafoSocial_entropia',
'medicamentosAntiDepressivo_mediaMovelPonterada'
]


atributos_anterioes = ['caracteresOrientais_variancia', 
'emojis_variancia', 
'midia_variancia', 'midia_mediaMovelPonterada', 
'curtidas_media', 'curtidas_variancia', 'curtidas_mediaMovelPonterada',
'links_mediaMovelPonterada'
]

atributos_novos = ['hashtags_variancia', 'hashtags_mediaMovelPonterada', 
'retweets_variancia', 
'mencoes_variancia', 'mencoes_mediaMovelPonterada', 
'polaridade_entropia', 
'subjetividade_entropia'
]

df = pd.read_csv(f'datasets/twitterbase_{periodo}.csv', sep=';')

cv = KFold(n_splits=10, shuffle=True, random_state=42)
zscore = StandardScaler()

train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

label_encoder = LabelEncoder()
df['classe'] = label_encoder.fit_transform(df['classe']).astype('float64')

df1 = df[choq + ['classe']].copy()
df2 = df[choq + atributos_anterioes + ['classe']].copy()
df3 = df[choq + atributos_novos + ['classe']].copy()

x1 = df1.drop('classe', axis=1).copy()
y1 = df1['classe'].copy()
x1 = zscore.fit_transform(x1)

x2 = df2.drop('classe', axis=1).copy()
y2 = df2['classe'].copy()
x2 = zscore.fit_transform(x2)

x3 = df3.drop('classe', axis=1).copy()
y3 = df3['classe'].copy()
x3 = zscore.fit_transform(x3)

model1 = model_SimpleRNN()
model2 = model_LSTM()
model3 = model_Bidirectional()
model4 = model_CNN_LSTM()

x4 = df.drop('classe', axis=1).copy()
y4 = df['classe'].copy()
x4 = zscore.fit_transform(x4)

# Simulação 1

In [30]:
x_train, x_val, y_train, y_val = train_test_split(x1, y1, test_size=validation_ratio, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_ratio, random_state=42)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 1)

In [31]:
model1.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model1.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model1.evaluate(x_val, y_val, verbose=0)

print()
print('SimpleRNN')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


SimpleRNN
precisao 76.11
revocacao 73.25
f1-score 74.02
acuracia 74.02


In [32]:
model2.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model2.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model2.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM
precisao 64.81
revocacao 94.36
f1-score 76.46
acuracia 70.52


In [33]:
model3.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model3.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model3.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM Biredicional')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM Biredicional
precisao 70.98
revocacao 86.97
f1-score 77.72
acuracia 74.73


In [34]:
model4.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model4.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model4.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM + CNN')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM + CNN
precisao 70.87
revocacao 85.99
f1-score 77.22
acuracia 74.31


# Simulação 2

In [35]:
x_train, x_val, y_train, y_val = train_test_split(x2, y2, test_size=validation_ratio, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_ratio, random_state=42)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 1)

In [36]:
model1.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model1.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model1.evaluate(x_val, y_val, verbose=0)

print()
print('SimpleRNN')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


SimpleRNN
precisao 76.24
revocacao 76.40
f1-score 75.70
acuracia 75.14


In [37]:
model2.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model2.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model2.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM
precisao 75.80
revocacao 65.24
f1-score 69.33
acuracia 71.00


In [38]:
model3.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model3.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model3.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM Biredicional')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM Biredicional
precisao 70.93
revocacao 87.10
f1-score 77.76
acuracia 74.73


In [39]:
model4.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model4.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model4.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM + CNN')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM + CNN
precisao 69.17
revocacao 90.22
f1-score 77.88
acuracia 74.00


# Simulação 3

In [40]:
x_train, x_val, y_train, y_val = train_test_split(x3, y3, test_size=validation_ratio, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_ratio, random_state=42)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 1)

In [41]:
model1.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model1.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model1.evaluate(x_val, y_val, verbose=0)

print()
print('SimpleRNN')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


SimpleRNN
precisao 75.86
revocacao 78.44
f1-score 76.59
acuracia 75.71


In [42]:
model2.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model2.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model2.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM
precisao 68.27
revocacao 91.73
f1-score 77.81
acuracia 73.44


In [43]:
model3.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model3.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model3.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM Biredicional')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM Biredicional
precisao 72.32
revocacao 84.05
f1-score 77.26
acuracia 74.93


In [44]:
model4.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model4.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model4.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM + CNN')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM + CNN
precisao 72.34
revocacao 87.00
f1-score 78.53
acuracia 75.85


# Simulação 4

In [45]:
x_train, x_val, y_train, y_val = train_test_split(x4, y4, test_size=validation_ratio, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_ratio, random_state=42)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 1)

In [46]:
model1.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model1.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model1.evaluate(x_val, y_val, verbose=0)

print()
print('SimpleRNN')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


SimpleRNN
precisao 67.60
revocacao 91.51
f1-score 77.36
acuracia 72.84


In [47]:
model2.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model2.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model2.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM
precisao 79.28
revocacao 54.48
f1-score 63.65
acuracia 68.75


In [48]:
model3.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model3.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model3.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM Biredicional')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM Biredicional
precisao 78.67
revocacao 72.58
f1-score 74.96
acuracia 75.45


In [49]:
model4.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc', f1score, precision_m, recall_m])
model4.fit(x_train, y_train,  validation_data=(x_val, y_val),  epochs = epochs, batch_size = 128, shuffle=True, callbacks = [early_stop], verbose=0)
loss, acuracia, f1_score, precisao, revocacao = model4.evaluate(x_val, y_val, verbose=0)

print()
print('LSTM + CNN')
imprimir_metricas(precisao, revocacao, f1_score, acuracia)


LSTM + CNN
precisao 68.97
revocacao 94.14
f1-score 79.16
acuracia 74.86
